In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np
import pandas as pd
# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat


# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [76]:


# La entrada es de 28x28 elementos para las imágenes
input_layer_size = 28 * 28

# 10 etiquetas, de 0 a 9
num_labels = 10

# Cargar datos desde el archivo CSV, ignorando la primera fila que contiene los encabezados
data = np.loadtxt('/content/drive/MyDrive/laboratorios_IA/lab_5/maleModifiers.csv', delimiter=',', skiprows=1)
X = data[:, :-1] / 255.0  # Normalizar los valores de píxeles
y = data[:, -1]


#dataPandas=pd.DataFrame(data)
# Dividir los datos en características (X) y etiquetas (y)
#X = dataPandas.iloc[:, :-1]
#y = dataPandas.iloc[:, -1]
print(y)
print("las x : \n")
print(X)
print(data.shape)

#dataPandas.info()
#print(dataPandas)

# Normalizar los valores de los píxeles para que estén en el rango [0, 1]
#X = X / 255.0

# Establecer el dígito 10 en 0, ya que los datos de MATLAB asignan 10 para el dígito 0
#y[y == 11] = 0

#m = y.size


[0. 0. 0. ... 9. 9. 9.]
las x : 

[[0.         0.01176471 0.00784314 ... 0.         0.         0.        ]
 [0.00392157 0.00392157 0.01176471 ... 0.         0.         0.        ]
 [0.00392157 0.00392157 0.00392157 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.69019608 0.15686275 0.00784314]
 [0.         0.         0.         ... 0.01960784 0.01568627 0.        ]
 [0.1372549  0.2627451  0.60392157 ... 0.         0.         0.        ]]
(15189, 785)


In [80]:


class Perceptron:
    def __init__(self, num_features, regularization_lambda):
        self.weights = np.random.randn(num_features)
        self.bias = 0
        self.regularization_lambda = regularization_lambda

    def predict(self, x):
        z = np.dot(x, self.weights) + self.bias
        return self.sigmoid(z)

    def train(self, X, y, learning_rate=0.01, num_epochs=100):
        m = len(X)
        for _ in range(num_epochs):
            for i in range(m):
                prediction = self.predict(X[i])
                error = y[i] - prediction
                self.weights = (1 - 2 * learning_rate * self.regularization_lambda / m) * self.weights \
                               + learning_rate * error * X[i]
                self.bias += learning_rate * error

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))



print("mostrando los datos :")
print(data)

print("columna de y")
print(y)

# Establecer el dígito 10 en 0, ya que los datos de MATLAB asignan 10 para el dígito 0
y[y == 11] = 0

# Dividir los datos en entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir porcentaje de entrenamiento y prueba
print("Porcentaje de datos de entrenamiento:", len(X_train) / len(X) * 100, "%")
print("Porcentaje de datos de prueba:", len(X_test) / len(X) * 100, "%")

# Crear una instancia del perceptrón
perceptron_model = Perceptron(num_features=X_train.shape[1], regularization_lambda=0.001)

# Entrenar el modelo
perceptron_model.train(X_train, y_train, learning_rate=0.001, num_epochs=10)

# Evaluar el modelo en el conjunto de prueba
y_pred = perceptron_model.predict(X_test)
print("es l ay prediccha: ",y_pred)
print("es la y testeada: ",y_test)


# Calcular la precisión del modelo
accuracy = np.mean(y_pred == y_test)*100
print("Precisión del modelo:", accuracy)


mostrando los datos :
[[  0.   3.   2. ...   0.   0.   0.]
 [  1.   1.   3. ...   0.   0.   0.]
 [  1.   1.   1. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...  40.   2.   9.]
 [  0.   0.   0. ...   4.   0.   9.]
 [ 35.  67. 154. ...   0.   0.   9.]]
columna de y
[0. 0. 0. ... 9. 9. 9.]
Porcentaje de datos de entrenamiento: 79.99868325762066 %
Porcentaje de datos de prueba: 20.001316742379355 %
es l ay prediccha:  [1. 1. 1. ... 1. 1. 1.]
es la y testeada:  [8. 2. 9. ... 0. 6. 6.]
Precisión del modelo: 9.90783410138249


In [67]:
# Ajuste de hiperparámetros
learning_rates = [0.001, 0.01, 0.1]
num_epochs_list = [10, 20, 30]
regularization_lambda_list = [0.001, 0.01, 0.1]

best_accuracy = 0
best_hyperparameters = {}

for learning_rate in learning_rates:
    for num_epochs in num_epochs_list:
        for regularization_lambda in regularization_lambda_list:
            perceptron_model = Perceptron(num_features=X_train.shape[1], regularization_lambda=regularization_lambda)
            perceptron_model.train(X_train, y_train, learning_rate=learning_rate, num_epochs=num_epochs)
            y_pred = perceptron_model.predict(X_test)
            accuracy = np.mean(y_pred == y_test)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_hyperparameters = {
                    'learning_rate': learning_rate,
                    'num_epochs': num_epochs,
                    'regularization_lambda': regularization_lambda
                }

print("Mejor precisión encontrada:", best_accuracy)
print("Mejores hiperparámetros:", best_hyperparameters)





Mejor precisión encontrada: 0.09907834101382489
Mejores hiperparámetros: {'learning_rate': 0.001, 'num_epochs': 10, 'regularization_lambda': 0.001}


el modelo implementa un perceptrón, que es un tipo de red neuronal simple con una única capa. No hay capas ocultas en este modelo, ya que solo hay una capa de entrada y una capa de salida.

In [40]:
6188
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)


Dimensiones de X_train: (12151, 784)
Dimensiones de X_test: (3038, 784)
